In [1]:
import sys, csv, os
import numpy as np
import pandas as pd
import pdb

In [2]:
paths = {'trial1':'',
         'trial2':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\draft2\events.csv",
         'trial8444':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv",
         'trial4':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604882689569_35pc\instance_1\output\events.csv",
         'trial85a':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614483337038\instance_1\output\events.csv",
         'trial85b':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614485140334\instance_1\output\events.csv",
         'trial85c':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614526877900\instance_1\output\events.csv"
        }
#toy trial
#events_fpath = r"C:\Users\agutf\AppData\Local\Temp\simphony_model_1593922625316\instance_1\output\events.csv"
chosen_trial = 'trial8444'
#D2E60
events_fpath = paths[chosen_trial]

In [3]:
print("Reading events from: " + events_fpath)

Reading events from: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv


In [4]:
time_between_bleeds = 28
reporting_frequency = time_between_bleeds/365
#rebase_time_to_final_dose

In [5]:
print(f"Time between simulated bleeds (days): {time_between_bleeds}")

Time between simulated bleeds (days): 28


In [6]:
vaccine_trial_end_date = 1400/365.0 #external default

In [7]:
print(f"vaccine_trial_end_date (years): {vaccine_trial_end_date}")

vaccine_trial_end_date (years): 3.835616438356164


In [8]:
output_fpath = f'{events_fpath}_history_{chosen_trial}_gap{time_between_bleeds}.csv'
print(f"Output file: {output_fpath}")

Output file: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv_history_trial8444_gap28.csv


In [9]:
#events_file = open(events_file,'r')
#events = csv.DictReader(events_file)

In [10]:
#events_file.close()

In [11]:
trial_header = pd.read_csv(events_fpath, nrows=2, header=1)

In [12]:
#trial_header

In [13]:
burn_in_days = trial_header.burn_in_days[0]
vaccine_schedule = trial_header.vaccine_schedule[0]
vaccine_enrollment_launch_day = trial_header.vaccine_enrollment_launch_day


In [14]:
#vaccine_trial_end_date = 2*vaccine_trial_end_date

In [15]:
print(f'vaccine_schedule: {vaccine_schedule}')

vaccine_schedule: D2E60


In [16]:
events = pd.read_csv(events_fpath, header=4)
events.rename(columns={'Num Buddies':'total_network_size'}, inplace=True)

In [17]:
#events.tail()

In [18]:
#events[events.Event == 'enteredfollowup'].head()

In [19]:
#pd.unique(events.HCV)

In [20]:
trial_starts = events[events.Event == 'trialstarted'].copy()
trial_starts['RNA_at_start_of_trial'] = trial_starts.Info.str.contains('RNA=true')
trial_starts = trial_starts.drop(columns=['EC','L1','L2','N1','N2','C','Info','Event','DBLabel','HCV'])
trial_starts = trial_starts.rename(columns={'Time':'time_entered_trial'})

agent_baselines = {}
for row in trial_starts.iterrows():
    agent_data = row[1]
    agent = agent_data.Agent
    agent_baselines[agent] = agent_data

In [21]:
#trial_starts[trial_starts.Agent==254477147]

In [22]:
events2 = events[events.Agent.isin(agent_baselines)]

In [23]:
vaccinations = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'vaccinated')]
agent_vaccination_doses = vaccinations.groupby(['Agent']).size().to_dict()

In [24]:
agent_abpos_at_enrollment = {}
agent_vaccination_time = events2[(events2.Event == 'vaccinated')].groupby(['Agent'])['Time'].first().to_dict()
agent_vaccination_time = {a: float(v) for a, v in agent_vaccination_time.items()}
for event in events2.itertuples():
    if float(event.Time) < agent_vaccination_time[event.Agent]:
        agent_abpos_at_enrollment[event.Agent] = 'abneg' if event.HCV == 'susceptible' else 'abpos'

In [25]:
for agent in agent_baselines:
    agent_baselines[agent]['ABpos_at_start_of_trial'] = agent_abpos_at_enrollment[agent]
    agent_baselines[agent]['doses_received'] = agent_vaccination_doses[agent]

In [26]:
ever_infected = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'infected')]
infected_roster = ever_infected.groupby(['Agent']).size().to_dict()
#for agents with infections, count of the number of infections

In [27]:
#ever_recovered = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'recovered')]
#recovered_count = ever_recovered.groupby(['Agent']).size().to_dict()

In [28]:
events2[events2.Agent==254477147]

,Time,EC,Agent,Event,Info,L1,N1,L2,N2,C,...,current_total_network_size,Daily_injection_intensity,Fraction_recept_sharing,DBLabel,trial_arm,chicago_community_name,fraction_total_network_female,fraction_total_network_nh_white,fraction_total_network_inder30,mean_age_total_network


In [29]:
#events2[events2.Agent == list(agent_baselines.keys())[2]]

In [30]:
#pd.unique(events2.Event)

In [31]:
#events2[events2.Agent==1359870546].iloc[0]

In [32]:
#events2[events2.Agent==300460332]

In [33]:
#the following events do not provide new information about the RNA state or the censorship, so we drop them
# if the agent becomes infected or recovered, we will retrieve it from "infected" and "recovered" events below
#  remove to simplify the parsing
# retained infectious and chronic for safety's sake
events3 = events2[~events2.Event.isin
                  (['activated','vaccinated'])]

In [34]:
events3[events3.Agent==300460332]

,Time,EC,Agent,Event,Info,L1,N1,L2,N2,C,...,current_total_network_size,Daily_injection_intensity,Fraction_recept_sharing,DBLabel,trial_arm,chicago_community_name,fraction_total_network_female,fraction_total_network_nh_white,fraction_total_network_inder30,mean_age_total_network
33281,1.184,NaN,300460332,trialstarted,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.293151
35626,1.348,NaN,300460332,enteredfollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.457534
37229,1.425,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.534247
39019,1.501,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.610959
41032,1.578,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.687671
43232,1.655,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,1,7.052627,0.303813,3957,placebo,OutsideChicago,0.0,1.0,0.0,32.663014
45676,1.732,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,32.673973
48318,1.808,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,32.750685
51173,1.885,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,32.827397
54255,1.962,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,32.904110


In [35]:
#pd.unique(events3.Event)

In [36]:
events2[events2.Agent==300460332]

,Time,EC,Agent,Event,Info,L1,N1,L2,N2,C,...,current_total_network_size,Daily_injection_intensity,Fraction_recept_sharing,DBLabel,trial_arm,chicago_community_name,fraction_total_network_female,fraction_total_network_nh_white,fraction_total_network_inder30,mean_age_total_network
23094,1.000,NaN,300460332,activated,NaN,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,noarm,OutsideChicago,0.5,1.0,0.5,29.104110
33281,1.184,NaN,300460332,trialstarted,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.293151
33282,1.184,NaN,300460332,vaccinated,newstage=notenrolled;RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.293151
35625,1.348,NaN,300460332,vaccinated,newstage=notenrolled;RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.457534
35626,1.348,NaN,300460332,enteredfollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.457534
37229,1.425,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.534247
39019,1.501,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.610959
41032,1.578,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.5,28.687671
43232,1.655,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,1,7.052627,0.303813,3957,placebo,OutsideChicago,0.0,1.0,0.0,32.663014
45676,1.732,NaN,300460332,infollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,2,7.052627,0.303813,3957,placebo,OutsideChicago,0.5,1.0,0.0,32.673973


In [37]:
agent_events = {}
longest_history = 0 #needed for padding the data
for agent in agent_baselines:
    a_events = events3[events3.Agent == agent][['Event','HCV','Info','Time']]
    a_events = a_events[a_events.Time <= vaccine_trial_end_date]
    a_events = a_events.sort_values(by="Time") #just in case
    a_events = a_events.append(pd.DataFrame([['fieldworkcompleted', '-', vaccine_trial_end_date]], columns=list(['Event','HCV', 'Time'])))
    a_events["StepsFromEnrollmentStart"] = (a_events.Time - agent_baselines[agent]["time_entered_trial"])/reporting_frequency
    agent_events[agent] = a_events
    if len(a_events[a_events.Event=='enteredfollowup']) > 0:
        longest_history = max(longest_history, a_events.Time.iloc[-1]-a_events[a_events.Event=='enteredfollowup'].Time.iloc[0])  
                          #min(vaccine_trial_end_date-a_events.Time.iloc[0], 
                          #    a_events.Time.iloc[-1]-a_events.Time.iloc[0]))

longest_history_len = longest_history/reporting_frequency

In [38]:
a_events

,Event,HCV,Info,Time,StepsFromEnrollmentStart
59487,trialstarted,vaccinated,RNA=false;arm=study,2.082000,0.000000
66354,enteredfollowup,vaccinated,RNA=false;arm=study,2.244000,2.111786
69449,infollowup,vaccinated,RNA=false;arm=study,2.321000,3.115536
72407,infollowup,vaccinated,RNA=false;arm=study,2.397000,4.106250
75348,infollowup,vaccinated,RNA=false;arm=study,2.474000,5.110000
78243,infollowup,vaccinated,RNA=false;arm=study,2.551000,6.113750
81081,infollowup,vaccinated,RNA=false;arm=study,2.627000,7.104464
83881,infollowup,vaccinated,RNA=false;arm=study,2.704000,8.108214
86454,infollowup,vaccinated,RNA=false;arm=study,2.781000,9.111964
88814,infollowup,vaccinated,RNA=false;arm=study,2.858000,10.115714


In [39]:
agent_events[300460332]

,Event,HCV,Info,Time,StepsFromEnrollmentStart
33281,trialstarted,vaccinated,RNA=false;arm=placebo,1.184000,0.000000
35626,enteredfollowup,vaccinated,RNA=false;arm=placebo,1.348000,2.137857
37229,infollowup,vaccinated,RNA=false;arm=placebo,1.425000,3.141607
39019,infollowup,vaccinated,RNA=false;arm=placebo,1.501000,4.132321
41032,infollowup,vaccinated,RNA=false;arm=placebo,1.578000,5.136071
43232,infollowup,vaccinated,RNA=false;arm=placebo,1.655000,6.139821
45676,infollowup,vaccinated,RNA=false;arm=placebo,1.732000,7.143571
48318,infollowup,vaccinated,RNA=false;arm=placebo,1.808000,8.134286
51173,infollowup,vaccinated,RNA=false;arm=placebo,1.885000,9.138036
54255,infollowup,vaccinated,RNA=false;arm=placebo,1.962000,10.141786


In [40]:
#print(longest_history)
#longest_history_len = longest_history/reporting_frequency
#print(longest_history_len)

#for agent, a_events in agent_events.items():
#    enrollment_start = agent_events[agent].Time.iloc[0]
#    if a_events['Time'].iloc[-1] < longest_history:
#        agent_events[agent] = agent_events[agent].append(pd.DataFrame([['padding', 'padding', longest_history]], columns=list(['Event','HCV', 'Time'])))
#    agent_events[agent].sort_values(by="Time", inplace=True)

In [41]:
#agent=list(agent_events.keys())[1]

In [42]:
#agent_events[agent]

In [43]:
#pd.unique(events3.Event)

In [44]:
#pd.unique(events3.HCV)

In [62]:
'''
produce a sequence [+,-,-,...,lost,lost,...,padding,padding,..]
the history records all infection episodes through the end of the trial or loss with padding
'''
def estimate_history(agent_id, my_events, target_length):
    h = [] #test the correct count [current_hcv]
    last_hcv_reading = None
    final_status = 'padding'
    if my_events.iloc[0].Event != 'trialstarted':
        print(f'WARNING: history for agent {agent_id} does not contain trialstarted event')
        return h, 'Unknown'
    for new_event in my_events.itertuples():
        if new_event.Event in frozenset(['enteredfollowup','infollowup','infollowup2']): #involve bleeding
            last_hcv_reading = 'RNA_positive' if 'RNA=true' in new_event.Info else 'RNA_negative'
            h = h + [last_hcv_reading]
        elif new_event.Event in frozenset(['trialabandoned', 'deactivated', 'fieldworkcompleted']):
            final_status = 'lost_to_followup'
            h = h + [final_status]
            break
        elif new_event.Event in frozenset(['trialcompleted']): #does not involve bleeding (19th event, normally)
            break
        elif new_event.Event in frozenset(['trialstarted', 'infected', 'infectious', 'recovered', 'chronic']):
            continue
        else:
            print(f'WARNING: history for agent {agent_id} has unknown event {new_event.Event}')
    
    h = h + [final_status]*int(target_length-len(h))
    
    #pdb.set_trace()
    return h, last_hcv_reading

In [46]:
#agent_events[300460332]

In [63]:
#agent=300460332
#estimate_history(agent,
#                 agent_events[agent], 
#                 longest_history_len)

In [48]:
#estimate_history(list(agent_events.values())[2], 
#                 agent_baselines[agent]['RNA_at_start_of_trial'], 
#                 longest_history_len)

In [49]:
#list(agent_events.values())[2]

In [64]:
agent_histories = {}
for agent, a_events in agent_events.items():
    regular_history, last_hcv_reading = estimate_history(agent, 
                                                         a_events, 
                                                         longest_history_len)
    if regular_history == []: #very rare - e.g. rarely first event is "deactivated" (bug in APK)
        continue
    
    agent_baselines[agent]['notinfected_and_abpos_at_final_reading']   = \
               (agent_baselines[agent]['trial_arm'] == 'study' and last_hcv_reading == 'RNA_negative') or \
               (agent_baselines[agent]['trial_arm'] == 'placebo' and (agent in infected_roster) and last_hcv_reading == 'RNA_negative')
    #print(agent_baselines[agent]['notinfected_and_abpos_at_final_reading'])
    #print(agent_baselines[agent]['trial_arm'])
    #print(agent in infected_count)
    #print(last_hcv_reading)
    
    agent_histories[agent] = agent_baselines[agent].append(pd.Series(regular_history))

In [65]:
print('total agents parsed: %d'%len(agent_histories))
#events3[events3.Agent==1623552776]

total agents parsed: 3207


In [52]:
#agent_histories[1623552776]

In [53]:
#agent_histories[agent]

In [54]:
#a_events

In [55]:
#a_events

In [56]:
#agent_histories[agent]

In [57]:
#vaccine_trial_end_date

In [66]:
output_df = pd.DataFrame.from_dict(agent_histories, orient='columns')

In [67]:
#agent_0 = agent_histories.keys()[0]
#output_df.set_index(agent_baselines[agent_0].keys() + range(longest_history), inplace=True)

In [68]:
#output_df = output_df.transpose()

In [69]:
output_df.to_csv(output_fpath)

In [70]:
print(f'Written: {output_fpath}')

Written: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv_history_trial8444_gap28.csv


-----